In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from aml_storage import Indexes, Block, Descriptor

from aml_rascal_bridge import RascalSphericalExpansion

import ase.io

In [2]:
frames = ase.io.read("data/molecule_conformers_dftb.xyz", ":2")

In [3]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.5,
    "max_radial": 6,
    "max_angular": 6,
    "gaussian_sigma_type": "Constant",
    "compute_gradients": True,
}

calculator = RascalSphericalExpansion(rascal_hypers)

In [4]:
descriptor = calculator.compute(frames)

In [5]:
print(descriptor.sparse_indexes.names)
print(descriptor.sparse_indexes[:30:5])

('spherical_harmonics_l', 'center_species', 'neighbor_species')
[(0, 1, 1) (0, 6, 8) (1, 1, 6) (1, 8, 1) (2, 1, 8) (2, 8, 6)]


In [6]:
# we can also look for in Indexes as named tuples

for count, index in enumerate(descriptor.sparse_indexes.as_namedtuples()):
    if count > 6:
        break
    print(index)
    print(index.as_dict())
    print()

IndexTuple(spherical_harmonics_l=0, center_species=1, neighbor_species=1)
{'spherical_harmonics_l': 0, 'center_species': 1, 'neighbor_species': 1}

IndexTuple(spherical_harmonics_l=0, center_species=1, neighbor_species=6)
{'spherical_harmonics_l': 0, 'center_species': 1, 'neighbor_species': 6}

IndexTuple(spherical_harmonics_l=0, center_species=1, neighbor_species=8)
{'spherical_harmonics_l': 0, 'center_species': 1, 'neighbor_species': 8}

IndexTuple(spherical_harmonics_l=0, center_species=6, neighbor_species=1)
{'spherical_harmonics_l': 0, 'center_species': 6, 'neighbor_species': 1}

IndexTuple(spherical_harmonics_l=0, center_species=6, neighbor_species=6)
{'spherical_harmonics_l': 0, 'center_species': 6, 'neighbor_species': 6}

IndexTuple(spherical_harmonics_l=0, center_species=6, neighbor_species=8)
{'spherical_harmonics_l': 0, 'center_species': 6, 'neighbor_species': 8}

IndexTuple(spherical_harmonics_l=0, center_species=8, neighbor_species=1)
{'spherical_harmonics_l': 0, 'center_s

In [7]:
block = descriptor.block(
    spherical_harmonics_l=4, 
    center_species=1, 
    neighbor_species=1,
)

# order is samples, symmetric, features
print(block.values.shape)

(12, 9, 6)


In [8]:
print(block.samples.names)
print(block.samples[:6])

('structure', 'center')
[(0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9)]


In [9]:
print(block.symmetric.names)
print(block.symmetric)

('spherical_harmonics_m',)
[(-4,) (-3,) (-2,) (-1,) ( 0,) ( 1,) ( 2,) ( 3,) ( 4,)]


In [10]:
print(block.features.names)
print(block.features)

('n',)
[(0,) (1,) (2,) (3,) (4,) (5,)]


In [11]:
gradients_samples, gradients = block.gradient("positions")

print(gradients.shape)

print(gradients_samples.names)
print(gradients_samples[:10])

(156, 9, 6)
('sample', 'atom', 'spatial')
[(0, 4, 0) (0, 4, 1) (0, 4, 2) (0, 9, 0) (0, 9, 1) (0, 9, 2) (0, 6, 0)
 (0, 6, 1) (0, 6, 2) (0, 5, 0)]


In [12]:
# since there is a single oxygen atom, there are no contribution to the gradient
# with center_specie=8, neighbor_species=8, spherical_harmonics_l>0
block = descriptor.block(
    spherical_harmonics_l=4, 
    center_species=8, 
    neighbor_species=8,
)

gradients_samples, gradients = block.gradient("positions")
assert len(gradients) == 0

# Moving indexes around (from sparse to dense storage)

In [13]:
rascal_hypers["compute_gradients"] = False

calculator = RascalSphericalExpansion(rascal_hypers)
descriptor = calculator.compute(frames)

descriptor.sparse_to_features("neighbor_species")

In [14]:
block = descriptor.block(center_species=1, spherical_harmonics_l=6)

block.values.shape

(12, 13, 18)

In [15]:
print(block.samples.names)
print(block.samples)

('structure', 'center')
[(0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9) (1, 4) (1, 5) (1, 6) (1, 7)
 (1, 8) (1, 9)]


In [16]:
try:
    descriptor.sparse_to_features("spherical_harmonics_l")
except Exception as e:
    print(e)

invalid parameter: can not move sparse index to features if the blocks have different symmetric indexes, call symmetric_to_features first


In [17]:
# we need to move the m index to features before moving l to features
descriptor.symmetric_to_features()

descriptor.sparse_to_features("spherical_harmonics_l")

In [18]:
block = descriptor.block(center_species=1)
block.values.shape

(12, 1, 882)

In [19]:
descriptor.sparse_to_samples("center_species")

In [20]:
# we now only have one block

block = descriptor.block()
block.values.shape

(20, 1, 882)

## Checking vs librascal storage

In [21]:
calculator = RascalSphericalExpansion(rascal_hypers)
descriptor = calculator.compute(frames)

descriptor.sparse_to_features("neighbor_species")
species_radial_size = descriptor.block(spherical_harmonics_l=0, center_species=1).values.shape[2]

descriptor.symmetric_to_features()
descriptor.sparse_to_features("spherical_harmonics_l")
descriptor.sparse_to_samples("center_species")

block = descriptor.block()
n_features = block.features.shape[0]

full_dense = block.values

# # put lm at the end of the features
full_dense = full_dense.reshape(full_dense.shape[0], -1, species_radial_size)
full_dense = full_dense.swapaxes(1, 2)
full_dense = full_dense.reshape(full_dense.shape[0], -1)

In [22]:
from rascal.representations import SphericalExpansion

rascal_calculator = SphericalExpansion(**rascal_hypers)
managers = rascal_calculator.transform(frames)
rascal_spx = managers.get_features(rascal_calculator)

In [23]:
assert np.all(rascal_spx == full_dense)